# Metodo de Newton para solucionar el Solver de Markovitz

In [0]:
import numpy as np
import cupy as cp
import solver.extraer_datos_yahoo as extrae
import solver.funciones_auxiliares as aux
import solver.line_search as line
import solver.modelo_markowitz as mkv
import solver.utils as utils
import solver.optimizacion_numerica as opt
#from utils import inc_index, dec_index, compute_error, norm_residual, condicion_cupy
#from line_search import line_search_by_backtracking, line_search_for_residual_by_backtracking

In [0]:
import solver

def Newtons_method_feasible_init_point(f, A, x_0, tol, 
                                       tol_backtracking, x_ast=None, p_ast=None, maxiter=30,
                                       gf_symbolic = None,
                                       Hf_symbolic = None,
                                       Sigma = None):
    '''
    Newton's method to numerically approximate solution of min f subject to Ax = b.
    IMPORTANT: this implementation requires that initial point x_0, satisfies: Ax_0 = b
    Args:
        f (fun): definition of function f as lambda expression or function definition.
        A (numpy ndarray): 2d numpy array of shape (m,n) defines system of constraints Ax=b.
        x_0 (numpy ndarray): initial point for Newton's method. Must satisfy: Ax_0 = b
        tol (float): tolerance that will halt method. Controls stopping criteria.
        tol_backtracking (float): tolerance that will halt method. Controls value of line search by backtracking.
        x_ast (numpy ndarray): solution of min f, now it's required that user knows the solution...
        p_ast (float): value of f(x_ast), now it's required that user knows the solution...
        maxiter (int): maximum number of iterations
        gf_symbolic (fun): definition of gradient of f. If given, no approximation is
                                     performed via finite differences.
        Hf_symbolic (fun): definition of Hessian of f. If given, no approximation is
                                     performed via fi
                                     nite differences.
    Returns:
        x (numpy ndarray): numpy array, approximation of x_ast.
        iteration (int): number of iterations.
        Err_plot (numpy ndarray): numpy array of absolute error between p_ast and f(x) with x approximation
                          of x_ast. Useful for plotting.
        x_plot (numpy ndarray): numpy array that containts in columns vector of approximations. Last column
                        contains x, approximation of solution. Useful for plotting.
    '''
    iteration = 0
        
    x = x_0
    
    feval = f(x)
    
    if gf_symbolic:
        Sigma = Sigma
        gfeval = gf_symbolic(x)
    else:
        gfeval = solver.diferenciacion_numerica.gradient_approximation(f,x)

    if Hf_symbolic:
        Sigma=Sigma
        Hfeval = Hf_symbolic(x)
    else:
        Hfeval = solver.diferenciacion_numerica.Hessian_approximation(f,x)
    
    normgf = np.linalg.norm(gfeval)
    condHf= solver.utils.condicion_cupy(Hfeval)
    
    Err_plot_aux = np.zeros(maxiter)
    Err_plot_aux[iteration]= solver.utils.compute_error(p_ast,feval)
    
    Err = solver.utils.compute_error(x_ast,x)
    
        
    if(A.ndim == 1):
        p = 1
        n = x.size
        zero_matrix = cp.zeros(p)
        first_stack = cp.column_stack((Hfeval, A.T))
        second_stack = cp.row_stack((A.reshape(1,n).T,zero_matrix)).reshape(1,n+1)[0]
    else:
        p,n = A.shape
        zero_matrix = cp.zeros((p,p))
        first_stack = np.column_stack((Hfeval, A.T))
        second_stack = np.column_stack((A,zero_matrix))
        
    x_plot = cp.zeros((n,maxiter))
    x_plot[:,iteration] = x
    
    system_matrix = np.row_stack((first_stack,second_stack))
    zero_vector = cp.zeros(p)
    rhs = np.row_stack((gfeval.reshape(n,1), zero_vector.reshape(p,1))).T[0]

    #Newton's direction and Newton's decrement
    dir_desc = cp.linalg.solve(system_matrix, -rhs)
    dir_Newton = dir_desc[0:n]
    dec_Newton = -gfeval.dot(dir_Newton)
    w_dual_variable_estimation = dir_desc[n:(n+p)]


    print('I\tNormgf \tNewton Decrement\tError x_ast\tError p_ast\tline search\tCondHf')
    print('{}\t{}\t{}\t{}\t{}\t{}\t\t{}'.format(iteration,
                                                normgf,
                                                dec_Newton,
                                                Err,
                                                Err_plot_aux[iteration],"---",
                                                condHf))
    stopping_criteria = dec_Newton/2
    iteration+=1
    while(stopping_criteria>tol and iteration < maxiter):
        der_direct = -dec_Newton
        t = solver.line_search.line_search_by_backtracking(f,dir_Newton,x,der_direct)
        x = x + t*dir_Newton
        feval = f(x)
        
        
        if gf_symbolic:
            Sigma = Sigma
            gfeval = gf_symbolic(x)
        else:
            gfeval = solver.diferenciacion_numerica.gradient_approximation(f,x)
        
        if Hf_symbolic:
            Sigma = Sigma
            Hfeval = Hf_symbolic(x)
        else:
            Hfeval = solver.diferenciacion_numerica.Hessian_approximation(f,x)
        if(A.ndim == 1):
            first_stack = cp.column_stack((Hfeval, A.T))
        else:
            first_stack = cp.column_stack((Hfeval, A.T))

        system_matrix = np.row_stack((first_stack,second_stack))
        rhs = np.row_stack((gfeval.reshape(n,1), zero_vector.reshape(p,1))).T[0]
        #Newton's direction and Newton's decrement
        dir_desc = cp.linalg.solve(system_matrix, -rhs)
        dir_Newton = dir_desc[0:n]
        dec_Newton = -gfeval.dot(dir_Newton)
        w_dual_variable_estimation = dir_desc[n:(n+p)]
        
        Err_plot_aux[iteration]= solver.utils.compute_error(p_ast,feval)
        x_plot[:,iteration] = x
        Err = solver.utils.compute_error(x_ast,x)
        print('{}\t{}\t{}\t{}\t{}\t{}\t{}'.format(iteration,
                                                  normgf,
                                                  dec_Newton,Err,
                                                  Err_plot_aux[iteration],t,
                                                  cp.asarray(condHf)))
        stopping_criteria = dec_Newton/2
        if t<tol_backtracking: #if t is less than tol_backtracking then we need to check the reason
            iter_salida=iteration
            iteration = maxiter - 1
        iteration+=1
    print('{} {}'.format("Error of x with respect to x_ast:",Err))
    print('{} {}'.format("Approximate solution:", x))
    cond = Err_plot_aux > np.finfo(float).eps*10**(-2)
    Err_plot = Err_plot_aux[cond]
    
    if iteration == maxiter and t < tol_backtracking:
        print("Backtracking value less than tol_backtracking, check approximation")
        iteration=iter_salida
    else:
        if iteration == maxiter:
            print("Reached maximum of iterations, check approximation")
    x_plot = x_plot[:,:iteration]
    return [x,iteration,Err_plot,x_plot]

## Cargamos la información y calculamos los parámetros

In [0]:
stocks = ['COP','AMT','LIN','LMT','AMZN','WMT','JNJ','VTI','MSFT','GOOG','XOM','CCI','BHP.AX','UNP',
'BABA','NSRGY','RHHBY','VOO','AAPL','FB','CVX','PLD','RIO.L','HON','HD','PG','UNH','BRK-A','V','0700.HK',
'RDSA.AS','0688.HK','AI.PA','RTX','MC.PA','KO','PFE','JPM','005930.KS','VZ','RELIANCE.NS','DLR','2010.SR',
'UPS','7203.T','PEP','MRK','1398.HK','MA','T']

In [3]:
datos = extrae.extraer_datos_yahoo(stocks)

[*********************100%***********************]  50 of 50 downloaded


In [0]:
mu = aux.calcular_rendimiento(datos)

In [0]:
S = aux.calcular_varianza(datos)

In [6]:
r=max(mu).item()
r

0.4022108787760788

## Resolvemos con el Método de Newton usando diferencias finitas

In [0]:
fo = lambda w: w@S@w

In [0]:
w_ast = mkv.markowitz(r,mu,S, df=False)

In [9]:
n = mu.shape[0]
A = cp.concatenate((mu,cp.ones(n))).reshape(2,n)
A

array([[ 0.13254293,  0.02782955,  0.25778345, -0.01787111, -0.02008539,
        -0.02286483,  0.19078421,  0.05743478,  0.17062275,  0.40221088,
         0.13618598,  0.02364206,  0.0476638 ,  0.13217291, -0.09666644,
        -0.03426582,  0.15880965,  0.17870289,  0.18557566,  0.15042008,
         0.08379133,  0.07154137,  0.08843336,  0.02202303,  0.07353272,
         0.13731909,  0.23605478,  0.19942876,  0.06806557,  0.26292508,
         0.07357148,  0.06803817,  0.03867589,  0.05090149,  0.14401416,
        -0.09339291,  0.23072551,  0.05036842,  0.05587177, -0.01264577,
        -0.01474117,  0.20678446,  0.06274014, -0.02729424,  0.1990038 ,
         0.07054765,  0.06563628,  0.04203765,  0.0717407 , -0.13227261],
       [ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.

In [10]:
b = cp.array([r,1])
b

array([0.40221088, 1.        ])

Definimos el punto inicial como:

In [0]:
M = cp.ones((2,mu.shape[0]))

In [0]:
w_0 = utils.feasible_markowitz(r,mu)

In [0]:
tol=1e-8
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=50

In [14]:
[x,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)



I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0019178915263183885	0.000887556410867015	1.1002919607342356	4.696439257807226	---		10050.199593921367
1	0.0019178915263183885	3.760062932545962e-08	0.04238267306445035	0.00019325422894986887	1	10050.199593921367
2	0.0019178915263183885	6.474617300015255e-11	0.003025598335118704	3.520979637982383e-07	1	10050.199593921367
Error of x with respect to x_ast: 0.003025598335118704
Approximate solution: [ 1.58428546e-01 -2.08616309e-02  1.58020949e-01 -9.28935082e-02
  3.16991935e-02  5.10945480e-02  8.72922956e-02  1.26346166e-02
  3.73103284e-02  2.86016263e-01 -5.94665541e-03  2.23126418e-03
  2.03110573e-01  9.45805956e-02  2.28799709e-02  1.50331910e-02
  7.62589108e-03  2.97057955e-02  5.77183696e-02  1.98175883e-01
  1.19874923e-01  1.27950342e-01  1.41431816e-01  1.36290060e-02
  8.84466664e-02  1.50547999e-01  1.69210858e-01  7.72068032e-02
  8.09578029e-02  8.25608723e-02  1.92191572e-01 -2.38661332e-02
  2.647

In [15]:
x

array([ 1.58428546e-01, -2.08616309e-02,  1.58020949e-01, -9.28935082e-02,
        3.16991935e-02,  5.10945480e-02,  8.72922956e-02,  1.26346166e-02,
        3.73103284e-02,  2.86016263e-01, -5.94665541e-03,  2.23126418e-03,
        2.03110573e-01,  9.45805956e-02,  2.28799709e-02,  1.50331910e-02,
        7.62589108e-03,  2.97057955e-02,  5.77183696e-02,  1.98175883e-01,
        1.19874923e-01,  1.27950342e-01,  1.41431816e-01,  1.36290060e-02,
        8.84466664e-02,  1.50547999e-01,  1.69210858e-01,  7.72068032e-02,
        8.09578029e-02,  8.25608723e-02,  1.92191572e-01, -2.38661332e-02,
        2.64704653e-02,  7.70401324e-02,  2.20865617e-02, -1.08224222e-01,
        1.64026211e-01,  1.77103658e-02,  6.16445527e-02, -1.08192451e-01,
       -5.03004393e-02,  1.38878495e-01,  1.03603985e-01, -4.27834232e-02,
        1.08153614e-02, -2.33911126e+00,  1.94282246e-01,  2.63290259e-01,
        1.20150533e-01, -1.05386379e-01])

In [16]:
w_ast

array([ 1.58450459e-01, -2.08710279e-02,  1.58051613e-01, -9.28970852e-02,
        3.17093838e-02,  5.10824537e-02,  8.71800626e-02,  1.26389547e-02,
        3.72861469e-02,  2.86000507e-01, -5.98810147e-03,  2.24206232e-03,
        2.03075836e-01,  9.46030741e-02,  2.28766788e-02,  1.49919976e-02,
        7.60706433e-03,  2.96676402e-02,  5.76521006e-02,  1.98109863e-01,
        1.19928144e-01,  1.27869501e-01,  1.41300419e-01,  1.36285005e-02,
        8.83904753e-02,  1.50479914e-01,  1.69293512e-01,  7.72037012e-02,
        8.09121352e-02,  8.24658724e-02,  1.92197879e-01, -2.40095431e-02,
        2.64375219e-02,  7.69647088e-02,  2.20741648e-02, -1.08207562e-01,
        1.64032748e-01,  1.77020932e-02,  6.16398087e-02, -1.08210745e-01,
       -5.03462807e-02,  1.38834320e-01,  1.03567400e-01, -4.28198353e-02,
        1.06872745e-02, -2.33314798e+00,  1.89811544e-01,  2.63296457e-01,
        1.20089773e-01, -1.05535600e-01])

In [17]:
x@S@x

array(9.45979579e-05)

In [18]:
w_ast@S@w_ast

array(9.45979246e-05)

In [19]:
x@mu

array(0.40221088)

In [20]:
w_ast@mu

array(0.40221088)

In [21]:
sum(x)

array(1.)

## Resolvemos con el Método de Newton usando funciones simbólicas